<a href="https://colab.research.google.com/github/sau-rabh999/Deep-Learning/blob/main/Word2Vec_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
#importing libraries

import pandas as pd
import numpy as np
from numpy import array
import string
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

Source Text

In [117]:
para = '''
Reed College at that time offered perhaps the best calligraphy instruction in the country. Throughout the campus every poster, every label on every drawer, was beautifully hand calligraphed. Because I had dropped out and didn’t have to take the normal classes, I decided to take a calligraphy class to learn how to do this. I learned about serif and sans serif typefaces, about varying the amount of space between different letter combinations, about what makes great typography great. It was beautiful, historical, artistically subtle in a way that science can’t capture, and I found it fascinating.
None of this had even a hope of any practical application in my life. But 10 years later, when we were designing the first Macintosh computer, it all came back to me. And we designed it all into the Mac. It was the first computer with beautiful typography. If I had never dropped in on that single course in college, the Mac would have never had multiple typefaces or proportionally spaced fonts. And since Windows just copied the Mac, it’s likely that no personal computer would have them. If I had never dropped out, I would have never dropped in on this calligraphy class, and personal computers might not have the wonderful typography that they do. Of course it was impossible to connect the dots looking forward when I was in college. But it was very, very clear looking backward 10 years later.
Again, you can’t connect the dots looking forward; you can only connect them looking backward. So you have to trust that the dots will somehow connect in your future. You have to trust in something — your gut, destiny, life, karma, whatever. This approach has never let me down, and it has made all the difference in my life.
When I was 17, I read a quote that went something like: “If you live each day as if it was your last, someday you’ll most certainly be right.” It made an impression on me, and since then, for the past 33 years, I have looked in the mirror every morning and asked myself: “If today were the last day of my life, would I want to do what I am about to do today?” And whenever the answer has been “No” for too many days in a row, I know I need to change something.
Remembering that I’ll be dead soon is the most important tool I’ve ever encountered to help me make the big choices in life. Because almost everything — all external expectations, all pride, all fear of embarrassment or failure, these things just fall away in the face of death, leaving only what is truly important. Remembering that you are going to die is the best way I know to avoid the trap of thinking you have something to lose. You are already naked. There is no reason not to follow your heart.

'''

Preprocessing

In [118]:
import re
text = re.sub(r'\[[0-9]*\]',' ',para)
text = re.sub(r'\s+',' ',text)
text = text.lower()
text = re.sub(r'\d',' ',text)

In [119]:
text

' reed college at that time offered perhaps the best calligraphy instruction in the country. throughout the campus every poster, every label on every drawer, was beautifully hand calligraphed. because i had dropped out and didn’t have to take the normal classes, i decided to take a calligraphy class to learn how to do this. i learned about serif and sans serif typefaces, about varying the amount of space between different letter combinations, about what makes great typography great. it was beautiful, historical, artistically subtle in a way that science can’t capture, and i found it fascinating. none of this had even a hope of any practical application in my life. but    years later, when we were designing the first macintosh computer, it all came back to me. and we designed it all into the mac. it was the first computer with beautiful typography. if i had never dropped in on that single course in college, the mac would have never had multiple typefaces or proportionally spaced fonts. 

In [120]:
len(text)

2694

# Predicting next words

integer encode data

In [121]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])                            #Update internal vocabulary based on a list of text
encoded_data = tokenizer.texts_to_sequences([text])[0]    #Transforms each word in text to a sequence of integers
encoded_data

[85,
 30,
 86,
 7,
 87,
 88,
 89,
 1,
 42,
 31,
 90,
 4,
 1,
 91,
 92,
 1,
 93,
 17,
 94,
 17,
 95,
 18,
 17,
 96,
 9,
 97,
 98,
 99,
 43,
 2,
 13,
 19,
 44,
 5,
 100,
 8,
 3,
 45,
 1,
 101,
 102,
 2,
 103,
 3,
 45,
 14,
 31,
 46,
 3,
 104,
 105,
 3,
 20,
 21,
 2,
 106,
 22,
 47,
 5,
 107,
 47,
 48,
 22,
 108,
 1,
 109,
 10,
 110,
 111,
 112,
 113,
 114,
 22,
 32,
 115,
 49,
 33,
 49,
 6,
 9,
 50,
 116,
 117,
 118,
 4,
 14,
 51,
 7,
 119,
 52,
 120,
 5,
 2,
 121,
 6,
 122,
 123,
 10,
 21,
 13,
 124,
 14,
 125,
 10,
 126,
 127,
 128,
 4,
 34,
 15,
 53,
 35,
 54,
 36,
 55,
 56,
 129,
 1,
 57,
 130,
 37,
 6,
 12,
 131,
 132,
 3,
 23,
 5,
 55,
 133,
 6,
 12,
 134,
 1,
 38,
 6,
 9,
 1,
 57,
 37,
 135,
 50,
 33,
 39,
 2,
 13,
 16,
 19,
 4,
 18,
 7,
 136,
 58,
 4,
 30,
 1,
 38,
 24,
 8,
 16,
 13,
 137,
 48,
 59,
 138,
 139,
 140,
 5,
 60,
 141,
 61,
 142,
 1,
 38,
 143,
 144,
 7,
 62,
 63,
 37,
 24,
 8,
 64,
 39,
 2,
 13,
 16,
 19,
 44,
 2,
 24,
 8,
 16,
 19,
 4,
 18,
 21,
 31,
 46,
 5,
 63,


Determine the vocab size

In [122]:
vocab_size = len(tokenizer.word_index)+1    #0 reserved for padding
print('Vocabulary size is: ',vocab_size)

Vocabulary size is:  236


**Creation of sequences of words to fit the model with 1 word as input and 1 word as output (input --> output)**

In [123]:
sequences = list()
for i in range(1,len(encoded_data)):
  sequence = encoded_data[i-1:i+1]
  sequences.append(sequence)
print('Total sequences: ',len(sequences))  


Total sequences:  482


In [124]:
sequences[:20]

[[85, 30],
 [30, 86],
 [86, 7],
 [7, 87],
 [87, 88],
 [88, 89],
 [89, 1],
 [1, 42],
 [42, 31],
 [31, 90],
 [90, 4],
 [4, 1],
 [1, 91],
 [91, 92],
 [92, 1],
 [1, 93],
 [93, 17],
 [17, 94],
 [94, 17],
 [17, 95]]

Splitting the sequences into input elements(x) and output elements(y)

In [125]:
sequences = array(sequences)
x,y = sequences[:,0],sequences[:,1]

In [126]:
x[:10]

array([85, 30, 86,  7, 87, 88, 89,  1, 42, 31])

In [127]:
y[:10]

array([30, 86,  7, 87, 88, 89,  1, 42, 31, 90])

One hot encode outputs

In [128]:
y = to_categorical(y , num_classes=vocab_size)
y[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Building the model

In [129]:
model = Sequential()
model.add(Embedding(vocab_size , 20 , input_length=1)) #(max_features, embedding_size, input_length)
model.add(LSTM(50))
model.add(Dense(vocab_size , activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 20)             4720      
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                14200     
_________________________________________________________________
dense_2 (Dense)              (None, 236)               12036     
Total params: 30,956
Trainable params: 30,956
Non-trainable params: 0
_________________________________________________________________
None


Compile the network

In [130]:
model.compile(loss='categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])


Fit network

In [131]:
model.fit(x,y, epochs=100)

Epoch 1/100
16/16 [==============================] - 2s 3ms/step - loss: 5.4632 - accuracy: 0.0062 
Epoch 2/100
16/16 [==============================] - 0s 3ms/step - loss: 5.4562 - accuracy: 0.0471
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 5.4497 - accuracy: 0.0618
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 5.4420 - accuracy: 0.0682
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 5.4335 - accuracy: 0.0625
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 5.4197 - accuracy: 0.0749
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 5.4020 - accuracy: 0.0881
Epoch 8/100
16/16 [==============================] - 0s 3ms/step - loss: 5.3738 - accuracy: 0.0721
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 5.3233 - accuracy: 0.1031
Epoch 10/100
16/16 [==============================] - 0s 3ms/step - loss: 5.2844 - accuracy: 0.0766
Epoch 11

Generate a sequence from the model

In [132]:
def generate_seq(model, tokenizer, in_text, n_pred):
  result = in_text

  #generate a fixed no of words
  for _ in range(n_pred):

    #encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = array(encoded)

    #predict a word in the vocabulary
    y_pred = model.predict_classes(encoded)

    #map the predicted word index to the word
    out_word = ''
    for word,index in tokenizer.word_index.items():  #for each of the item/word, we will check if it matches with the predicted word y_pred
      if index==y_pred:
        out_word = word
        break

    #append to input
    in_text,result = out_word, result+' '+out_word
  return result

Evaluate next n words

In [133]:
print(generate_seq(model,tokenizer,'offered',2))

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


offered perhaps the


In [134]:
print(generate_seq(model,tokenizer,'death',5))

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


death leaving only connect the last


In [135]:
print(generate_seq(model,tokenizer,'failure',5))

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


failure these things just copied the


# find similar words (using gensim)

In [136]:
import nltk
import numpy
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [137]:
tokens = word_tokenize(text)

In [138]:
model2= Word2Vec([tokens],min_count=1)

In [139]:
vocabulary = model2.wv.vocab

Checking most similar words


In [140]:
sim_words = model2.wv.most_similar('failure')

In [141]:
sim_words 

[('this', 0.29643937945365906),
 ('between', 0.21947115659713745),
 ('encountered', 0.2030329406261444),
 ('how', 0.19071707129478455),
 ('“', 0.18395422399044037),
 ('that', 0.18068420886993408),
 ('everything', 0.1719420701265335),
 ('wonderful', 0.16356709599494934),
 ('trust', 0.1625773012638092),
 ('or', 0.150347501039505)]

In [142]:
sim_words = model2.wv.most_similar('great')

In [143]:
sim_words

[('looking', 0.4612054228782654),
 ('most', 0.24055829644203186),
 ('live', 0.23015420138835907),
 ('encountered', 0.2161312699317932),
 ('fall', 0.2070375233888626),
 ('follow', 0.19644944369792938),
 ('make', 0.19485414028167725),
 ('hand', 0.19178980588912964),
 ('has', 0.18897981941699982),
 ('quote', 0.18821923434734344)]

Checking similarity between 2 words

In [144]:
score = model2.wv.similarity('fonts', 'typography')
score

0.08615618

In [145]:
score = model2.wv.similarity('reed', 'college')
score

-0.17955215

In [146]:
cosine_similarity = numpy.dot(model2['fonts'], model2['typography'])
cosine_similarity

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


7.092531e-05